# Deconvolutions

Demonstration of deconvolutions.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

Init Plugin
Init Graph Optimizer
Init Kernel


## Example 1: Direct convolution and back

We can implement the convolution by flattening the kernel over the positions on the input. This yields a matrix of shape $(4,16)$ in the example. We then also have to flatten the input to get a shape of $(16,1)$. The dot product then has the shapes $(4,16)$ X $(16,1) = (4,1)$.
Finally, the output has to be reshaped into shape $(2,2)$.

The direct convolution:
* Input size: $(4,4)$
* Output size: $(2,2)$
* Kernel size: $(3,3)$
* Stride: $(1,1)$

The corresponding transposed convolution:
* Input size: $(2,2)$
* Output size: $(4,4)$
* Kernel size: $(3,3)$
* Stride: $(1,1)$


In [2]:
# Transposing the matrix explicitly

## =========================================================
# Forward / convolution
# Make input (4,4), flatten it to shape (16,1).
x = np.ones([1,4,4,1])
x = x.reshape(16,1)

# Flatten the convolution operation. Stride (1,1) with the kernel (3,3) yields 4 positions.
C = np.asarray([[1,1,1,0,1,1,1,0,1,1,1,0,0,0,0,0],
                [0,1,1,1,0,1,1,1,0,1,1,1,0,0,0,0],
                [0,0,0,0,1,1,1,0,1,1,1,0,1,1,1,0],
                [0,0,0,0,0,1,1,1,0,1,1,1,0,1,1,1]
                ])

# Compute the output.
y = np.dot(C, x)
print("Output shape direct convolution: ", y.shape)

## =========================================================
# Backward / deconvolution, transposed convolution
# Make input and flatten it to shape (4,1).
x = np.asarray([[9,9],[9,9]])
x = x.reshape(4,1)

# Transpose the matrix C.

C_t = C.transpose()
# Compute the output.
y = np.dot(C_t, x)
print("Output shape deconvolution / transposed convolution: ", y.shape)

Output shape direct convolution:  (4, 1)
Output shape deconvolution / transposed convolution:  (16, 1)


We can implement the same in tensorflow with `Conv2D` and `Conv2DTranspose`. We can set the kernel weights to 1 to replicate the example from above.

Note that we need to provide a channel dimension and we also provide a batch dimension here.

Looking at the values of the outputs we can see that they are the same as above. In the forward case the result is 

```
[[9,9],
[9,9]]
```

and in the backward case it is

```
[[9,18,18,9],
[18,36,36,18],
[18,36,36,18],
[9,18,18,9]].

In [4]:
# Direct convolution from (4,4) -> (2,2)

# Make input.
x = np.ones([1,4,4,1])
x = tf.convert_to_tensor(x, dtype=tf.float16)
x = tf.reshape(x,[1,4,4,1]) # Add batch and channel dimension
print("Input shape: ", x.shape)

# Convolve.
model = tf.keras.Sequential([
    layers.Conv2D(1, (3,3), strides=(1,1), padding='valid', kernel_initializer='ones', use_bias=False)
    ])
y = model(x)

tf.print("Output shape: ", y.shape)
#print(y) # Uncomment to view the output values

Input shape:  (1, 4, 4, 1)
Output shape:  TensorShape([1, 2, 2, 1])


In [5]:
# Deconvolution / Transposed convolution from (2,2) -> (4,4)

# Make input.
x = np.asarray([[9,9],[9,9]])
x = tf.convert_to_tensor(x, dtype=tf.float16)
x = tf.reshape(x,[1,2,2,1])
print("Input shape: ", x.shape)

# Convolve
model = tf.keras.Sequential([
    layers.Conv2DTranspose(1,(3,3),strides=(1,1), padding='valid', kernel_initializer='ones', use_bias=False)
])
y = model(x)

tf.print("Output shape: ", y.shape)
#print(y) # Uncomment to view the output values

Input shape:  (1, 2, 2, 1)
Output shape:  TensorShape([1, 4, 4, 1])


## Example 2: Deconvolution as convolution on padded input

We can also implement the deconvolution from $(2,2) -> (4,4)$ by padding the input with $0s$ on the boarder up to shape $(6,6)$ and a direct convolution. That way, the model can still learn the upsampling as opposed to upsampling directly from $(2,2) -> (4,4)$.


In [6]:
# Deconvolution as convolution with padded input

# Make input.
x = np.asarray([[9,9],[9,9]])
x = tf.convert_to_tensor(x, dtype=tf.float16)
x = tf.reshape(x,[1,2,2,1])
print("Input shape: ", x.shape)

# Pad input with 0s.
paddings = tf.constant([[0,0],[2,2],[2,2],[0,0]])
x = tf.pad(x, paddings=paddings)
print("Padded input shape: ", x.shape)

# Convolve.
model = tf.keras.Sequential([
    layers.Conv2D(1, (3,3), strides=(1,1), padding='valid', kernel_initializer='ones', use_bias=False)
])
y = model(x)

tf.print("Output shape: ", y.shape)
#print(y) # Uncomment to view the output values

Input shape:  (1, 2, 2, 1)
Padded input shape:  (1, 6, 6, 1)
Output shape:  TensorShape([1, 4, 4, 1])


As we can see, this yields the same result as in example 1 where we used `Conv2DTranspose`. We can also implement this by flattening the input and the kernel positions. However, since the matrix $C$ is of shape $(16,144)$ we won't write it out here.

# Example 3: Fractionally strided convolution as direct convolution
What if we want to go to a different output size, say $(2,2) -> (5,5)$? Here we can pad the input with 0s on the boarder as well as between elements, which transforms the input to shape $(7,7)$.

This operation shows why a deconvolution is also called fractionally strided convolution. To move the kernel from one cell from the original input to its neighbour in the padded input, a stride of $2$ instead of $1$ is required. Thus a stride of $1$ corresponds to a stride of $1/2$.

In [7]:
# Direct convolution plus padding between elements and on border

# Make input, with padding between elements as well as on the border.
x = np.asarray([[9,0,9],[0,0,0],[9,0,9]])
x = x.reshape(1,3,3,1) # Add batch and channel dimension
x = tf.convert_to_tensor(x, dtype=tf.float16)
paddings = tf.constant([[0,0],[2,2],[2,2],[0,0]])
x = tf.pad(x, paddings=paddings)
print("Padded input shape: ", x.shape)

# Convolve.
model = tf.keras.Sequential([
    layers.Conv2D(1, (3,3), strides=(1,1), padding='valid', kernel_initializer='ones', use_bias=False)
])

y = model(x)
tf.print("Output shape: ", y.shape)
#print(y) # Uncomment to view the output values

Padded input shape:  (1, 7, 7, 1)
Output shape:  TensorShape([1, 5, 5, 1])


In [8]:
# Transposed convolution
x = np.asarray([[9,9],[9,9]])
x = x.reshape(1,2,2,1)
print("Input shape: ", x.shape)
x = tf.convert_to_tensor(x, dtype=tf.float16)

model = tf.keras.Sequential([
    layers.Conv2DTranspose(1, (3,3), strides=(2,2), padding='valid', kernel_initializer='ones', use_bias=False)
])

y = model(x)
tf.print("Output shape: ", y.shape)
#print(y)

Input shape:  (1, 2, 2, 1)
Output shape:  TensorShape([1, 5, 5, 1])
